# 토크나이저

## [1] 라이브러리 임포트 & 토크나이저 로드

In [1]:
from transformers import BertTokenizer
from datasets import Dataset

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

## [2] 사전 처리 함수 정의 (text → input_ids, attention_mask)

In [2]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512  # BERT 입력 최대 길이
    )

## [3] 데이터셋 HuggingFace 형식으로 변환

In [3]:
import os
import json
import pandas as pd

# 경로 설정
BASE_DIR = os.path.abspath(os.path.join(".."))
LABEL_CSV_PATH = os.path.join(BASE_DIR, "data", "gpt4.1_label_1000d.csv")
LABEL_MAP_PATH = os.path.join(BASE_DIR, "data", "label_map.json")
RAW_JSON_DIR = os.path.join(BASE_DIR, "data", "raw", "원본")

# 라벨 맵 로딩
with open(LABEL_MAP_PATH, "r", encoding="utf-8") as f:
    label_map = json.load(f)

# CSV 로딩
df_label = pd.read_csv(LABEL_CSV_PATH)


In [4]:
from collections import defaultdict
from tqdm import tqdm
import re

def extract_target_text(full_text: str, fallback_len: int = 1000) -> str:
    party_info = re.search(r"(【원고, 피상고인】.*?【피고, 상고인】)", full_text, re.DOTALL)
    judgement_info = re.search(r"(【주[\s]*문】.*?)(【이[\s]*유】|【이유】)", full_text, re.DOTALL)

    combined = ""
    if party_info:
        combined += party_info.group(1).strip() + "\n"
    if judgement_info:
        combined += judgement_info.group(1).strip()

    return combined.strip() if combined.strip() else full_text[:fallback_len].strip()

# 전처리 실행
data_list = []
for _, row in tqdm(df_label.iterrows(), total=len(df_label)):
    filename = str(row["파일명"]).strip()
    label_str = row["라벨링결과"].strip()
    label = label_map.get(label_str, None)

    json_path = os.path.join(RAW_JSON_DIR, filename)
    if not os.path.exists(json_path) or label is None:
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        case_data = json.load(f)

    full_text = case_data.get("판례내용", "")
    if not full_text:
        continue

    trimmed_text = extract_target_text(full_text)
    if trimmed_text:
        data_list.append({
            "text": trimmed_text,
            "label": label
        })

# 라벨 3(각하), 6(취하) 제외
filtered_data_list = [x for x in data_list if x["label"] not in [3, 6]]


100%|██████████| 843/843 [00:28<00:00, 29.09it/s]


In [12]:
print(filtered_data_list[0]['text'].replace('<br/>', '\n'))

【원고, 피상고인】 원고 (소송대리인 변호사 정장훈)
【피고, 상고인】
【주    문】
  원심판결 중 일실수입에 관한 피고들 패소 부분을 파기하고, 그 부분 사건을 서울지방법원 본원 합의부에 환송한다.
  피고들의 나머지 상고를 각 기각한다.
  상고가 기각된 부분에 대한 상고비용은 피고들의 부담으로 한다.




In [5]:
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(
    filtered_data_list,
    test_size=0.1,
    random_state=42,
    stratify=[x["label"] for x in filtered_data_list]
)

train, val = train_test_split(
    train_val,
    test_size=2/9,
    random_state=42,
    stratify=[x["label"] for x in train_val]
)


In [6]:
train_dataset = Dataset.from_list(train)
val_dataset = Dataset.from_list(val)
test_dataset = Dataset.from_list(test)

## [4] 토크나이징 적용

In [7]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/586 [00:00<?, ? examples/s]

Map:   0%|          | 0/168 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

In [8]:
print(tokenized_train[0])

{'text': '【주    문】<br/>원심판결의 본소에 관한 지연손해금 부분 중 피고(반소원고)에 대하여 132,928,990원에 대한 2001. 2. 27.부터 2007. 2. 27.까지는 연 5%, 그 다음날부터 갚는 날까지는 연 20%의 각 비율에 의한 금원을 초과하여 지급을 명한 피고(반소원고) 패소 부분을 파기하고, 그 부분에 해당하는 원심에서 교환적으로 변경된 원고(반소피고)의 지연손해금 청구를 기각한다.   원심판결의 반소에 관한 지연손해금 부분을 파기하여 그 부분 제1심판결을 취소하고, 그에 해당하는 피고(반소원고)의 청구를 기각한다.  각 나머지 상고를 기각한다.  소송총비용은 본소, 반소를 통틀어 각 50%씩 부담한다.<br/><br/>', 'label': 4, 'input_ids': [101, 1894, 9689, 9297, 1895, 133, 33989, 120, 135, 9612, 71013, 33323, 74322, 10459, 9358, 22333, 10530, 42300, 9706, 25486, 119053, 14523, 40032, 9365, 37712, 9694, 9946, 11664, 113, 9321, 22333, 14279, 11664, 114, 9559, 68357, 18385, 117, 75340, 117, 55291, 108280, 18154, 10272, 119, 123, 119, 10365, 119, 9365, 21876, 10202, 119, 123, 119, 10365, 119, 8939, 32815, 9568, 126, 110, 117, 8924, 52292, 41919, 17655, 8856, 11018, 8985, 18382, 11018, 9568, 10197, 110, 9637, 8844, 9379, 119183, 10530, 60804, 8928, 51684, 9757, 11882, 13374, 9706, 37568, 10622, 9281, 11102, 9946, 11664, 113, 9321, 2